<a href="https://colab.research.google.com/github/illliyas/Machine-Learning/blob/master/tweets/Live_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic
!pip install joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Lets import important libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bertopic import BERTopic

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = pd.read_csv('ek_tweets7.csv')

In [73]:
df.head()

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
0,3.278147e+09,2022-10-06 01:48:35+00:00,,1.580000e+18,en,0,0,0,0,Twitter for Android,"@EmiratesSupport Thanks for the support, hope ..."
1,1.304480e+08,2022-10-06 02:34:23+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport Have replied to message. Plea...
2,1.693349e+08,2022-10-06 04:37:50+00:00,,1.580000e+18,en,0,0,0,0,Twitter Web App,@EmiratesSupport @EmiratesSupport i have DM'd ...
3,4.855336e+08,2022-10-06 04:41:22+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport sent you a dm Janine
4,6.556422e+07,2022-10-06 06:29:02+00:00,,1.580000e+18,en,0,0,1,0,Twitter for iPhone,@EmiratesSupport I have been mailing for fligh...


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   author id      934 non-null    float64
 1   created_at     934 non-null    object 
 2   geo            934 non-null    object 
 3   id             934 non-null    float64
 4   lang           934 non-null    object 
 5   like_count     934 non-null    int64  
 6   quote_count    934 non-null    int64  
 7   reply_count    934 non-null    int64  
 8   retweet_count  934 non-null    int64  
 9   source         934 non-null    object 
 10  tweet          934 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 80.4+ KB


In [5]:
df = df.drop_duplicates(subset='tweet', keep="first")

In [77]:
df.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 903 entries, 0 to 933
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   author id      903 non-null    float64
 1   created_at     903 non-null    object 
 2   geo            903 non-null    object 
 3   id             903 non-null    float64
 4   lang           903 non-null    object 
 5   like_count     903 non-null    int64  
 6   quote_count    903 non-null    int64  
 7   reply_count    903 non-null    int64  
 8   retweet_count  903 non-null    int64  
 9   source         903 non-null    object 
 10  tweet          903 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 84.7+ KB


In [6]:
list(df['tweet'][106:120])

['@EmiratesSupport I booked my flight from HYD-EWR for 3rd Nov using Student offer... I just wanted to know if I can merge the extra 10kg baggage allowance as per student offer in the luggage itself or do I need to carry an extra baggage of 10kg separately. Please reply!',
 '@EmiratesSupport I have shared details in ur DM',
 '@EmiratesSupport I have now given up.  Possibly easy to take this to a small claims court when I return to Australia',
 '@EmiratesSupport Finally, almost 45 days after my bag was broken, I have an answer: please, wait 20 days more for a "personalise response". Seriously? @EmiratesSupport @emirates @facua https://t.co/UQmSUTN443',
 '@EmiratesSupport See you this Friday for the return trip back. 🙂',
 'RT @fatman73: @EmiratesSupport @emirates @EmiratesSupport ticket cancelled on 07th Oct. Still no word from @emirates on refund amount and r…',
 '@EmiratesSupport The reply is useless I cannot submit the form. I paid over £3000 for my ticket and was delayed over 24 hour

In [17]:
stopwords

<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>

In [7]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [8]:
def cleaning_tweets(tweet):
    temp = tweet.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #Removing punctuations
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #Filtering non-alphanumeric characters
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = re.sub("([a-z]*[0-9]+[a-z0-9]* )+"," ", temp)
    #remove 2 letter words
    temp = re.sub(r"\b[a-z]{2}\b"," ", temp)
    return temp

def remove_stopwords(tweet):
    words = word_tokenize(tweet)
    words = [word for word in words if word not in sw]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words) 
def number_of_words(tweet):
  return len(word_tokenize(tweet))     

In [9]:
df['tweet_clean'] = df.apply(lambda row: cleaning_tweets(row['tweet']),axis=1)
df['tweet_clean2'] = df.apply(lambda row: remove_stopwords(row['tweet_clean']),axis=1)
df['number_of_words'] = df.apply(lambda row: number_of_words(row['tweet_clean2']),axis=1)

In [10]:
df2 = df[df['number_of_words']>2]
df2=df2.reset_index()

In [11]:
list(df2.tweet)[106:120]

['@EmiratesSupport Hi I have a Partner Visa from UAQ FTZ. What is the new time limit to stay out of UAE for such Visa holders?',
 "@EmiratesSupport Hi. I tried to join your skywards program but filled in my name incorrectly and can't change it. I was told to use Live Chat but have been trying for almost 2 months and it is always busy and telling me to come back later and try again. Help please.",
 '@EmiratesSupport  \r\nHi, just wondering how much does Emirates charge for extra luggage per kg. I am traveling from Chennai to Ireland.',
 '@EmiratesSupport PLEASE would someone answer the phone in London????  I have been on hold for TOOOO LONG!!!',
 '@EmiratesSupport I’m a caregiver to my elderly sick mom who has cancer. Her wish is to do Umrah. To fly she needs a lay flat seat cuz she’s always in pain, tired &amp; needs to lie down. Need help getting an affordable lay flat seat in Business/First Class to get to Mecca. Expensive',
 '@EmiratesSupport hi, your pet form wasn’t working online.

In [12]:
list(df2.tweet_clean2)[106:120]

['partner visa uaq ftz new time limit stay uae visa holder',
 'tried join skywards program filled name incorrectly change told use live chat trying almost month always busy telling come back later try help please',
 'wondering much emirate charge extra luggage per traveling chennai ireland',
 'please would someone answer phone london hold toooo long',
 'caregiver elderly sick mom cancer wish umrah fly need lay flat seat cuz always pain tired amp need lie need help getting affordable lay flat seat business first class get mecca expensive',
 'pet form working online transport small cat dubai yangon route operated emirate bangkok airline thanks',
 'emailed complaint couple time successful reply',
 'worst customer service airline dealt answer refund request delayed lost baggage filing complaint department transportation next response',
 'sorry say mean emirate support people ask help ground staff dept emirate send email tweet arrive feb duty manager ground staff hyd dubai one help till tod

In [13]:
df2['created_at'] = pd.to_datetime(df2['created_at'])
tweets = df2.tweet_clean2.to_list()
timestamps  = df2.created_at.to_list()

In [14]:
model1 = BERTopic(verbose=True,embedding_model='all-MiniLM-L6-v2', min_topic_size= 6)
topics, _ = model1.fit_transform(tweets)

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

2022-11-19 19:58:07,562 - BERTopic - Transformed documents to Embeddings
2022-11-19 19:58:15,468 - BERTopic - Reduced dimensionality
2022-11-19 19:58:15,503 - BERTopic - Clustered reduced embeddings


In [15]:
model1.get_topic_info()

,Topic,Count,Name
0,-1,226,-1_flight_ticket_please_would
1,0,145,0_emirate_dubai_flight_complaint
2,1,112,1_bag_luggage_baggage_checked
3,2,48,2_refund_cancel_rebook_money
4,3,37,3_booking_call_help_ticket
5,4,35,4_complaint_customer_email_take
6,5,27,5_vaccination_vaccinated_pcr_test
7,6,26,6_lounge_business_class_access
8,7,22,7_sent_thanks_thank_prompt
9,8,22,8_flight_chat_trying_possible


In [16]:
df2['tweet_topics'] = topics
model1.topic_labels_

{-1: '-1_flight_ticket_please_would',
 0: '0_emirate_dubai_flight_complaint',
 1: '1_bag_luggage_baggage_checked',
 2: '2_refund_cancel_rebook_money',
 3: '3_booking_call_help_ticket',
 4: '4_complaint_customer_email_take',
 5: '5_vaccination_vaccinated_pcr_test',
 6: '6_lounge_business_class_access',
 7: '7_sent_thanks_thank_prompt',
 8: '8_flight_chat_trying_possible',
 9: '9_seat_able_select_selection',
 10: '10_meal_food_nut_cater',
 11: '11_message_response_reply_messaged',
 12: '12_please_check_urgent_kindly',
 13: '13_mile_skyward_balance_help',
 14: '14_book_trying_error_website',
 15: '15_name_surname_first_payment',
 16: '16_counter_early_check_reach',
 17: '17_upgrade_help_paid_grab',
 18: '18_dublin_exhausted_five_star'}

In [17]:
model1.visualize_topics()

In [18]:
model1.visualize_barchart()

In [23]:
topics_over_time = model1.topics_over_time(tweets, timestamps)

model1.visualize_topics_over_time(topics_over_time)

835it [00:06, 120.21it/s]


In [ ]:
type(model1.topic_labels_)

dict

In [24]:
topics_df = pd.DataFrame(model1.topic_labels_.items(), columns=['topic_id', 'topic_name'])

In [25]:
df2 = df2.merge(topics_df,how='left',left_on='tweet_topics',right_on='topic_id')

In [26]:
df2['retweet_count'] = df2['retweet_count'].astype(int)

In [27]:
df2[['retweet_count','tweet','topic_name']].sort_values('retweet_count',ascending=False)

,retweet_count,tweet,topic_name
157,9,RT @chrisfroome: @EmiratesSupport just tried t...,-1_flight_ticket_please_would
156,9,@EmiratesSupport just tried to check in for ou...,-1_flight_ticket_please_would
177,3,@EmiratesSupport good morning! Can you tell me...,10_meal_food_nut_cater
187,3,RT @DeborahKnipe: @EmiratesSupport good mornin...,10_meal_food_nut_cater
273,1,"@EmiratesSupport \r\nOk, so I had to cancel my...",2_refund_cancel_rebook_money
...,...,...,...
288,0,@EmiratesSupport @emirates @Emirates247 can an...,12_please_check_urgent_kindly
289,0,@EmiratesSupport is the Emirates pass gone? Th...,0_emirate_dubai_flight_complaint
290,0,@EmiratesSupport I have inboxed you regarding ...,3_booking_call_help_ticket
291,0,@EmiratesSupport cannot get an Indian visa bec...,0_emirate_dubai_flight_complaint


In [ ]:
topics_df

,topic_id,topic_name
0,-1,-1_flight_response_get_time
1,0,0_booking_change_help_flight
2,1,1_luggage_baggage_bag_emirate
3,2,2_complaint_customer_reply_email
4,3,3_please_sent_thank_reply
5,4,4_emirate_support_time_date
6,5,5_meal_nut_food_infant
7,6,6_vaccinated_test_vaccination_pcr
8,7,7_refund_rebook_cancelled_money
9,8,8_invoice_much_form_resolving


In [ ]:
list(df2[df2['topic_name']=='1_luggage_baggage_bag_emirate']['tweet'])

['@EmiratesSupport Hi, we are flying economy from LGW to DXB on 15th October. What are our luggage allowances? Your website says 23kg hold + 7kg in the cabin. My ticket from you just says 30kg with no reference to either hold or cabin and my ticket from my TA says 20kg. Thanks',
 '@EmiratesSupport I pay over £5K for business and the baggage is supposed to be a priority, and yet I’m still here and no bags 45 minutes later. Booked my driver for 8.45 and it’s almost 10pm! https://t.co/CwcoWlTTkb',
 '@EmiratesSupport are we able to add a pram in the hold for our 1yr old as part of our luggage allowance &amp; if so, can we hold on to it until boarding at the gate? London -&gt; Dubai &amp; vice versa',
 '@EmiratesSupport   Hi I am a frequent flyer of emirates airlines I traveled from emirate’s airlines on 18th of May to JFK USA on arrival I found my one of my checkin baggage broken.I complain to your emirates counter at JFK .They give me complain no JFKD550185.Till date not solve',
 '@Emirat

In [ ]:
df2.iloc[162]['tweet']

'@EmiratesSupport just tried to check in for our flight Saturday and your system has decided that my 4 and 6 year old should be sitting on opposite ends of the plane away from my wife and I 😵\u200d💫'

In [ ]:
df2.iloc[270]['tweet']

'@EmiratesSupport It doesn’t help. And I’m not happy being ripped off. Now what can you do? 🤣'